## Homework: Vector Search

In this homework, we will learn more about vector search
and embedding. Like in the module, we will use Qdrant and
fastembed

> It's possible that your answers won't match exactly. If it's the case, select the closest one.

Note: if you want to learn how vector search works under
the hood, check 
[homework 3 from 2024 cohort (questions 1-4)](../../2024/03-vector-search/homework.md)

## Embeddings

Qdrant uses [fastembed](https://github.com/qdrant/fastembed)
under the hood to turn text into vectors. 

We will now explore this library

Make sure it's installed:

```bash
pip install fastembed
```

Import it: 

```python
from fastembed import TextEmbedding
```


## Submit the results

* Submit your results here: https://courses.datatalks.club/llm-zoomcamp-2025/homework/hw2
* It's possible that your answers won't match exactly. If it's the case, select the closest one.


In [1]:
!pip install fastembed


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
from fastembed import TextEmbedding

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
import numpy as np

In [3]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

## Q1. Embedding the query

Embed the query: `'I just discovered the course. Can I join now?'`.
Use the `'jinaai/jina-embeddings-v2-small-en'` model. 

You should get a numpy array of size 512.

What's the minimal value in this array?

* -0.51
* -0.11
* 0
* 0.51

## Cosine similarity

The vectors that our embedding model returns are already normalized: their length is 1.0.

You can chech that by using the `norm` function:

```python
import numpy as np
np.linalg.norm(q)
```

Which means that we can simply compute the dot product between
two vectors to learn the cosine similarity between them.

For example, if you compute the cosine of the query vector with itself, the result will be 1.0:

```python
q.dot(q)
```

In [7]:
model_name = "jinaai/jina-embeddings-v2-small-en"

In [8]:
embedding_model = TextEmbedding(model_name=model_name)

In [9]:
TextEmbedding.list_supported_models()

[{'model': 'BAAI/bge-base-en',
  'sources': {'hf': 'Qdrant/fast-bge-base-en',
   'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en.tar.gz',
   '_deprecated_tar_struct': True},
  'model_file': 'model_optimized.onnx',
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.',
  'license': 'mit',
  'size_in_GB': 0.42,
  'additional_files': [],
  'dim': 768,
  'tasks': {}},
 {'model': 'BAAI/bge-base-en-v1.5',
  'sources': {'hf': 'qdrant/bge-base-en-v1.5-onnx-q',
   'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en-v1.5.tar.gz',
   '_deprecated_tar_struct': True},
  'model_file': 'model_optimized.onnx',
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.',
  'license': 'mit',
  'size_in_GB': 0.21,
  'additional_files': [],
  'dim': 768,
  'tasks': {}},
 {'model':

In [56]:
query = "I just discovered the course. Can I join now?"
array = embedding_model.query_embed(query)

In [57]:
min(array)[0]

np.float64(-0.07639462602911798)

In [35]:
X = np.array(array)
X

array(<generator object TextEmbedding.embed at 0x74a457ad4400>,
      dtype=object)

## Q2. Cosine similarity with another vector

Now let's embed this document:

```python
doc = 'Can I still join the course after the start date?'
```

What's the cosine similarity between the vector for the query
and the vector for the document?

* 0.3
* 0.5
* 0.7
* 0.9


In [41]:
query = "I just discovered the course. Can I join now?"
doc = "Can I still join the course after the start date?"

In [61]:
query_vec = np.array(list(embedding_model.embed(query)))[0]
doc_vec = np.array(list(embedding_model.embed(doc)))[0]

In [62]:
similarity = np.dot(query_vec, doc_vec)
similarity

np.float64(0.9008528895674548)

## Q3. Ranking by cosine

For Q3 and Q4 we will use these documents:

```python
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]
```

Compute the embeddings for the text field, and compute the 
cosine between the query vector and all the documents.

What's the document index with the highest similarity? (Indexing starts from 0):

- 0
- 1
- 2
- 3
- 4

Hint: if you put all the embeddings of the text field in one matrix `V` (a single 2-dimensional numpy array), then
computing the cosine becomes a matrix multiplication:

```python
V.dot(q)
```

If this hint is rather confusing you than helping, feel free
to ignore it.

In [66]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [74]:
V = np.array(list(embedding_model.embed(doc["text"] for doc in documents)))
V

array([[-0.02495248, -0.03964539, -0.00437672, ...,  0.06779867,
         0.0377203 , -0.00470725],
       [-0.05947089, -0.08523984,  0.01292088, ...,  0.09599708,
         0.05420842, -0.00029459],
       [-0.06846455, -0.04079096,  0.04998121, ...,  0.06578071,
         0.02872077, -0.01115215],
       [-0.04640506, -0.02555229,  0.02241225, ...,  0.03410312,
         0.06813593, -0.0064033 ],
       [-0.05394911, -0.04693813,  0.00794725, ...,  0.05402997,
         0.03033386, -0.01254517]], shape=(5, 512))

In [76]:
query = "I just discovered the course. Can I join now?"
query_vec = np.array(list(embedding_model.embed(query)))

In [78]:
cos_ranking = V.dot(query_vec[0])

In [79]:
closest = np.argmax(cos_ranking)

closest

np.int64(1)

## Q4. Ranking by cosine, version two

Now let's calculate a new field, which is a concatenation of
`question` and `text`:

```python
full_text = doc['question'] + ' ' + doc['text']
``` 

Embed this field and compute the cosine between it and the
query vector. What's the highest scoring document?

- 0
- 1
- 2
- 3
- 4

Is it different from Q3? If yes, why?

In [89]:
V2 = np.array(list(embedding_model.embed(doc["question"] + " " + doc["text"] for doc in documents)))
cos_ranking2 = V2.dot(query_vec[0])
closest2 = np.argmax(cos_ranking2)

print(closest2)

0


## Q5. Selecting the embedding model

Now let's select a smaller embedding model.
What's the smallest dimensionality for models in fastembed?

- 128
- 256
- 384
- 512

One of these models is `BAAI/bge-small-en`. Let's use it.

In [90]:
embedding_model2 = TextEmbedding(model_name="BAAI/bge-small-en")

Fetching 5 files: 100%|███████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  3.04it/s]


In [92]:
embedding_model2.get_embedding_size(model_name="BAAI/bge-small-en")

384

In [97]:
min_dim = 512
for models in TextEmbedding.list_supported_models():
    min_dim = min(min_dim, models["dim"])

print(min_dim)

384


In [98]:
min([model["dim"] for model in TextEmbedding.list_supported_models()])

384

## Q6. Indexing with qdrant (2 points)

For the last question, we will use more documents.

We will select only FAQ records from our ml zoomcamp:

```python
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
```

Add them to qdrant using the model form Q5.

When adding the data, use both question and answer fields:

```python
text = doc['question'] + ' ' + doc['text']
```

After the data is inserted, use the question from Q1 for querying the collection.

What's the highest score in the results?
(The score for the first returned record):

- 0.97
- 0.87
- 0.77
- 0.67




In [122]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []
for course in documents_raw:
    if course['course'] == 'machine-learning-zoomcamp':
        for doc in course['documents']:
            doc['course'] = course['course']
            documents.append(doc)

In [123]:
from fastembed import TextEmbedding
from qdrant_client import QdrantClient, models

In [125]:
model_handle="BAAI/bge-small-en"
embedding_model = TextEmbedding(model_name=model_handle)

In [127]:
collection_name = "zoomcamp-homework-w2"

In [133]:
client.delete_collection(collection_name=collection_name)

True

In [134]:
client = QdrantClient("http://localhost:6333")

client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=384,
        distance=models.Distance.COSINE
    )
)

True

In [136]:
points = []
for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = list(embedding_model.embed(text))[0]
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [137]:
query = 'I just discovered the course. Can I join now?'

In [108]:
def search(query, limit=1):
    result = client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=query,
            model=model_handle,
        ),
        limit=limit,
        with_payload=True
    )
    return result

In [146]:
query_vec = list(embedding_model.embed(query))[0]

results = client.search(
    collection_name=collection_name,
    query_vector=query_vec,
    limit=1,
    with_payload=True
)

print(results[0].score)

0.87031734


/tmp/ipykernel_21286/141823257.py:3: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(
